<a href="https://colab.research.google.com/github/AbrarHossainHimself/AbrarHossainHimself/blob/main/UCR_prog_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview

This tool captures all possible function call sequences for a given Python program. By analyzing the Abstract Syntax Tree (AST) of the program, it generates all potential call sequences that could occur during actual execution. The tool outputs a structured representation of these call paths, which can aid in understanding complex call flows, especially in recursive or interdependent function environments.

## Structure

The tool is organized into three main components:

1. **FunctionCallAnalyzer**: This class builds a call graph by visiting each function definition and call expression in the source code. Using `ast.NodeVisitor`, it identifies and stores the relationships between functions in the program.
  
2. **CallSequenceGenerator**: Based on the call graph, this class recursively generates all possible call sequences up to a specified depth (`max_depth`). It ensures each unique sequence is captured without redundancy, even in cases of recursion or mutual recursion.
  
3. **analyze_python_code**: This function orchestrates the process. It takes Python source code as input, parses it to create an AST, builds the call graph using `FunctionCallAnalyzer`, and generates all possible call sequences for each function using `CallSequenceGenerator`. The final output is a dictionary of unique sequences for each function, providing a detailed representation of all possible execution paths.

## Usage

To analyze a Python program's call sequences, use the `analyze_python_code` function. This function accepts a string containing the source code and returns a dictionary, where each key is a function name, and the value is a list of lists representing unique call sequences from that function.

### Example

```python
# Sample source code
source_code = """
def main():
    foo()
    bar()

def foo():
    baz()

def bar():
    baz()
    qux()

def baz():
    pass

def qux():
    baz()
"""

# Generate call sequences
sequences = analyze_python_code(source_code)

# Print the sequences
for function, call_sequences in sequences.items():
    print(f"\nPossible call sequences starting from '{function}':")
    for seq in call_sequences:
        print(" -> ".join(seq))
```

### Expected Output

The output will list all possible function call paths, starting from each function and showing the various sequences of calls that could occur. For example:

```
Possible call sequences starting from 'main':
main -> foo -> baz
main -> bar -> baz
main -> bar -> qux -> baz
```

## Implementation Details

- **AST Parsing**: The `ast.parse` function is used to convert source code into an AST, which allows structured traversal and inspection of function definitions and calls.
- **Recursive Call Generation**: The `CallSequenceGenerator` uses a recursive approach to explore all possible paths, limiting recursion depth to `max_depth` to prevent infinite loops in the case of deep or cyclic recursion.
- **Unique Sequence Filtering**: Duplicate call sequences are filtered out, ensuring that each call path is unique and free from redundancy.

## Constraints

- The tool currently handles functions referenced directly by name.
- Built-in and external library functions are included in sequences if directly called; however, they are not resolved to external definitions.
- Class methods and nested functions are not analyzed unless extended to do so.


In [21]:
import ast
from typing import Set, List, Dict
from collections import defaultdict

class FunctionCallAnalyzer(ast.NodeVisitor):
    def __init__(self):
        self.call_graph = defaultdict(list)
        self.current_function = None
        self.all_functions = set()

    def visit_FunctionDef(self, node):
        self.current_function = node.name
        self.all_functions.add(node.name)
        # Reset calls for this function
        self.call_graph[node.name] = []
        # Visit all nodes in the function body
        for stmt in node.body:
            self.visit(stmt)
        # Reset current_function after processing
        self.current_function = None

    def visit_Call(self, node):
        if isinstance(node.func, ast.Name):
            if self.current_function:
                # Only add if it's not already in the list
                if node.func.id not in self.call_graph[self.current_function]:
                    self.call_graph[self.current_function].append(node.func.id)
        self.generic_visit(node)

class CallSequenceGenerator:
    def __init__(self, call_graph: Dict[str, List[str]], all_functions: Set[str]):
        self.call_graph = call_graph
        self.all_functions = all_functions
        self.max_depth = 10

    def generate_sequences(self, start_function: str) -> List[List[str]]:
        sequences = []
        visited = set()
        self._generate_sequences(start_function, [], sequences, visited, 0)
        return sequences

    def _generate_sequences(self, current_function: str, current_path: List[str],
                            sequences: List[List[str]], visited: Set[str], depth: int):
        """Generate sequences recursively."""
        if depth > self.max_depth or current_function not in self.all_functions:
            return

        # Add current function to path
        current_sequence = current_path + [current_function]
        sequences.append(current_sequence[:])

        # Get all direct function calls from current function
        if current_function in self.call_graph:
            for called_function in self.call_graph[current_function]:
                # Create unique call signature
                call_sig = f"{current_function}->{called_function}"
                if call_sig not in visited and depth < self.max_depth:
                    visited.add(call_sig)
                    self._generate_sequences(called_function, current_sequence,
                                             sequences, visited, depth + 1)
                    visited.remove(call_sig)

def analyze_python_code(source_code: str) -> Dict[str, List[List[str]]]:
    # Parse code
    tree = ast.parse(source_code)

    # Analyze AST
    analyzer = FunctionCallAnalyzer()
    analyzer.visit(tree)

    # Generate call sequences for each function
    generator = CallSequenceGenerator(analyzer.call_graph, analyzer.all_functions)

    # Generate and store sequences for each function
    all_sequences = {}
    for function in analyzer.all_functions:
        sequences = generator.generate_sequences(function)
        # Remove duplicates while preserving order
        unique_sequences = []
        seen = set()
        for seq in sequences:
            seq_tuple = tuple(seq)
            if seq_tuple not in seen:
                seen.add(seq_tuple)
                unique_sequences.append(seq)
        all_sequences[function] = unique_sequences

    return all_sequences


In [22]:
# Test Cases

test_cases = {
    "Test Case 1": """
def start():
    a()
    b()

def a():
    c()

def b():
    c()

def c():
    pass
""",
    "Test Case 2": """
def factorial(n):
    if n > 1:
        return n * factorial(n - 1)
    else:
        return 1
""",
    "Test Case 3": """
def is_even(n):
    if n == 0:
        return True
    else:
        return is_odd(n - 1)

def is_odd(n):
    if n == 0:
        return False
    else:
        return is_even(n - 1)
""",
    "Test Case 4": """
def standalone():
    pass
""",
    "Test Case 5": """
def outer():
    def inner():
        pass
    inner()
""",
    "Test Case 6": """
def func_a():
    pass

def func_b():
    pass

def caller():
    func = func_a
    func()
""",
    "Test Case 7": """
def main():
    try:
        risky_function()
    except Exception as e:
        handle_error(e)

def risky_function():
    raise ValueError("An error occurred")

def handle_error(e):
    pass
""",
    "Test Case 8": """
class MyClass:
    def method_a(self):
        self.method_b()

    def method_b(self):
        pass

def start():
    obj = MyClass()
    obj.method_a()
""",
    "Test Case 9": """
def compute():
    step1()
    step2()
    step1()

def step1():
    pass

def step2():
    pass
"""
}


In [23]:
selected_test_case = "Test Case 9"  # Change this to select a different test case

# Get source code
source_code = test_cases[selected_test_case]

# Analyze Python code
sequences = analyze_python_code(source_code)

# Print results
print(f"Analyzing {selected_test_case}:\n")
for function, call_sequences in sorted(sequences.items()):
    print(f"\nPossible call sequences starting from '{function}':")
    for seq in call_sequences:
        print(" -> ".join(seq))


Analyzing Test Case 9:


Possible call sequences starting from 'compute':
compute
compute -> step1
compute -> step2

Possible call sequences starting from 'step1':
step1

Possible call sequences starting from 'step2':
step2
